Берём датасет на миллион объектов. Если не хватает памяти, то берём 100 000.

Нужно подобрать лучшую модель используя кросс-валидацию. Можно использовать поиск по сетке.

Нужно получить метрику RMSE меньше 0.87.

In [2]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [3]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [4]:
movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', encoding='latin-1',
                     header=None, names=['movieId', 'title', 'genres'])
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', encoding='latin-1',
                      header=None, names=['userId', 'movieId', 'rating', 'timestamp'])

<ipython-input-4-089767fe3144>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', encoding='latin-1',
<ipython-input-4-089767fe3144>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', encoding='latin-1',


In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [6]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [7]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
movies_with_ratings

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474
...,...,...,...,...,...,...
1000204,3952,"Contender, The (2000)",Drama|Thriller,5812,4,992072099
1000205,3952,"Contender, The (2000)",Drama|Thriller,5831,3,986223125
1000206,3952,"Contender, The (2000)",Drama|Thriller,5837,4,1011902656
1000207,3952,"Contender, The (2000)",Drama|Thriller,5927,1,979852537


In [9]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357261 sha256=e09ae3971c1a82b9fd231773c09535b8d16f8ba7b36c75ac259cfc22a76eb0df
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [21]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [11]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader) #принимает только три знания, uid, iid, r

In [13]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=1)


Попробуем **KNNWithMeans** с разными значениями похожести

> Базовый алгоритм совместной фильтрации, учитывающий средние оценки каждого пользователя.








In [52]:
#есть 4 варианта похожести:
#cousine Вычислить косинусное сходство между всеми парами пользователей (или элементов).
#msd Вычислите сходство среднеквадратической разницы между всеми парами пользователей (или элементов).
#pearson Вычислите коэффициент корреляции Пирсона между всеми парами пользователей (или элементов).
#pearson_baseline Вычислите (сокращенный) коэффициент корреляции Пирсона между всеми парами пользователей (или элементов),
#используя для центрирования базовые линии вместо средних значений.

algo = KNNWithMeans(k=50, sim_options={  #ищем средний рейтинг по пользователям, используем 50 соседей
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})

#sim_options - опции для подсчёта похожестей, можно с ними поэкспериментировать
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [53]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9359


0.9359144262058818

In [54]:
algo = KNNWithMeans(k=50, sim_options={  #ищем средний рейтинг по пользователям, используем 50 соседей
    'name': 'msd',
    'user_based': True  # compute  similarities between users
})

#sim_options - опции для подсчёта похожестей, можно с ними поэкспериментировать
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9260


0.9260237347646417

In [56]:
algo = KNNWithMeans(k=50, sim_options={  #ищем средний рейтинг по пользователям, используем 50 соседей
    'name': 'pearson',
    'user_based': True  # compute  similarities between users
})

#sim_options - опции для подсчёта похожестей, можно с ними поэкспериментировать
algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9153


0.9152963494969294

In [58]:
algo = KNNWithMeans(k=50, sim_options={  #ищем средний рейтинг по пользователям, используем 50 соседей
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})

#sim_options - опции для подсчёта похожестей, можно с ними поэкспериментировать
algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8886


0.8886086632713015

**Сравнение RMSE:**

Cousine - 0.9359 (разбиралось в вебинаре)

msd - 0.9260

pearson - 0.9153

pearson_baseline - 0.8886

Вывод: лучше всего использовать сокращенный коэффициент корреляции Пирсона между всеми парами пользователей




Используя базовый коэффициент корреляции пирсона, попробуем поменять количество соседей:

In [59]:
algo = KNNWithMeans(k=10, sim_options={
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})

algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.9095


0.9094829264026907

In [14]:
algo = KNNWithMeans(k=60, sim_options={
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})

algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8888


0.8887967864050452

In [15]:
algo = KNNWithMeans(k=40, sim_options={
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})

algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8889


0.888898724782826

Сравнение RMSE:

k=10: 0.9095

k=40: 0.8889

k=50: 0.8886

k=60: 0.8888

Оставляем k = 50

Попробуем другие алгоритмы KNN из библиотеки surpise.

**KNNWithZScore**:



> Базовый алгоритм совместной фильтрации, учитывающий нормализацию z-оценки каждого пользователя



In [22]:
algo = KNNWithZScore(k=50, sim_options={
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})

algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8870


0.8869800855348983

**KNNBaseline**



> Алгоритм совместной фильтрации, учитывающий базовый рейтинг.



In [23]:
algo = KNNBaseline(k=50, sim_options={
    'name': 'pearson_baseline',
    'user_based': True  # compute  similarities between users
})


algo.fit(trainset)

test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.8693


0.8693121059953896

KNNWithZScore: 0.8870

KNNBaseline: 0.8693

Ура! Пришли к необходимому значению ниже 0.87.